#Ejercicio 1: Función "rot"

##1. Introducción
<justify>En este cuaderno, vamos a realizar la rotación de vectores en el plano de manera secuencial haciendo uso del procesador CPU, y su versión en paralelo optimizada con CUDA [4].

El algoritmo que se desarrollará a continuación está basado en la función "**rot**" [1], perteneciente a la biblioteca BLAS de nivel 1 [2]. Dicha función procede a realizar las siguientes operaciones: </justify>

<center>$x_i= c * x_i + s * y_i$    [3]</center>
<center>$y_i = c * y_i - s * x_i$</center>

Siendo *x* e *y* los vectores en cuestión, y *s* y *c* dos números escalares.


##2. Armado del ambiente
Para el armado del ambiente, es necesario instalar CUDA [5].

In [ ]:
!pip install pycuda

##3. Desarrollo
Comenzamos, en primera instancia, con la solicitud de los parámetros:
* "**cantidad**" es la cantidad de elementos de ambos vectores *x* e *y*.
* "**escalar_c**" es el número *c* en la ecuación.
* "**escalar_s**" es el número *s* en la ecuación.

Por otra parte, en el código se realizará primero la versión en paralelo y luego la secuencial. De todos modos, se toman los tiempos en los momentos correspondientes, sin influir el orden en el que se presentan.

In [ ]:
#----- Usamos esta parte para que el usuario ingrese los parámetros -----
#@title 3.1 Parámetro de ejecución { vertical-output: true }

cantidad = 51000 #@param {type: "integer"}
escalar_c = 2 #@param {type:"number"}
escalar_s = 5 #@param {type:"number"}

# --------------------------------------------

#Chequeamos que los parámetros tengan un valor correcto
if cantidad <=0:
  raise Exception("La cantidad de elementos de los vectores tienen que ser mayores a cero.")
if escalar_c <0:
  raise Exception("El valor de 'c' tiene que ser mayor o igual a cero.")
if escalar_s <0:
  raise Exception("El valor de 's' tiene que ser mayor o igual a cero.")  

#Importamos los módulos necesarios
from datetime import datetime as dt
import numpy as np
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule


#CPU: empezamos a tomar el tiempo para el procesamiento
tiempo_total = dt.now()

#Definimos la función que transforma el tiempo en  milisegundos 
tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0


#CPU: definimos la memoria de los vectores
x_cpu = np.random.randn(cantidad)
x_cpu = x_cpu.astype(np.float32())
res_x_cpu = np.empty_like(x_cpu)

y_cpu = np.random.randn(cantidad)
y_cpu = y_cpu.astype(np.float32())
res_y_cpu = np.empty_like(y_cpu)

#GPU: reservamos la memoria GPU
x_gpu = cuda.mem_alloc(x_cpu.nbytes)
y_gpu = cuda.mem_alloc(y_cpu.nbytes)

#Copiamos la memoria a GPU
cuda.memcpy_htod(x_gpu, x_cpu)
cuda.memcpy_htod(y_gpu, y_cpu)

#CPU: definimos la función kernel que va a ejecutar GPU
module = SourceModule("""
__global__ void kernel_rot(int n, int c, int s, float *X, float *Y)
{
  int idx = threadIdx.x + blockIdx.x * blockDim.x;

  //Nos guardamos temporalmente los valores que vamos a reemplazar
  float xi = 0.0;
  float yi = 0.0;
  
  if(idx < n)
  {
    xi = X[idx];
    yi = Y[idx];

    X[idx] = c * xi + s * yi;
    Y[idx] = c * yi - s * xi;
  }
}
""")

#CPU: generar la función kernel
kernel = module.get_function("kernel_rot")


#CPU: se define el tamaño de las dimensiones
dim_hilo = 256
dim_bloque = np.int((cantidad + dim_hilo - 1) / dim_hilo)
print("Thread x: ", dim_hilo, ", Bloque x: ", dim_bloque)

#Comenzamos a tomar el tiempo de ejecución de GPU
tiempo_gpu = dt.now()

#GPU: Mandamos la función kernel
kernel(np.int32(cantidad), np.int32(escalar_c), np.int32(escalar_s), x_gpu, y_gpu, block=(dim_hilo, 1, 1), grid=(dim_bloque, 1, 1))

#Cortamos el tiempo de GPU
tiempo_gpu = dt.now() - tiempo_gpu

#CPU: copia el resultado desde la memoria de GPU
cuda.memcpy_dtoh(res_x_cpu, x_gpu)
cuda.memcpy_dtoh(res_y_cpu, y_gpu)

#CPU: realizamos la función "dot" y empezamos a tomar el tiempo de ejecución del bucle
tiempo_bucle_cpu = dt.now()

for idx in range(0, cantidad):
  xi = x_cpu[idx]
  yi = y_cpu[idx]

  x_cpu[idx] = escalar_c * xi + escalar_s * yi
  y_cpu[idx] = escalar_c * yi - escalar_s * xi


tiempo_bucle_cpu = dt.now() - tiempo_bucle_cpu


#CPU: Tomamos el tiempo de ejecución total
tiempo_total = dt.now() - tiempo_total

"""
print("Resultado CPU:")
print("X: ", x_cpu)
print("Y: ", y_cpu)
print("-------------------------")
print("Resultado GPU:")
print("X: ", res_x_cpu)
print("Y: ", res_y_cpu)
print("-------------------------")
"""

#CPU: Informamos los tiempos de ejecución
print("Tiempo total: ", tiempo_en_ms(tiempo_total), "ms")
print("Tiempo del bucle en CPU: ", tiempo_en_ms(tiempo_bucle_cpu), "ms")
print("Tiempo de GPU: ", tiempo_en_ms(tiempo_gpu), "ms")

##4. Tabla de pasos
A continuación, realizaremos la tabla de pasos de ejecución del programa.

<center>

Procesador | Función | Detalle
--- | --- | ---
CPU | !pip install pycuda | Instalación de CUDA en el notebook actual
CPU | @param | Lectura del tamaño de los vectores *x* e *y*, y de los escalares *c* y *s* desde el formulario de Colab
CPU | if cantidad <=0 ... | Chequea que los parámetros sean válidos para el programa
CPU | raise Exception ... | Lanza excepciones en el caso de fallar la condición anterior
CPU | import | Importa los módulos necesarios para la ejecución del programa
CPU | dt.now() | Toma el tiempo actual a modo de tiempo inicial de ejecución
CPU | np.random.randn() | Inicializa los vectores *x* e *y*
CPU | np.empty_like() | Generación de los arrays necesarios para el resultado de los vectores
**GPU** | cuda.mem_alloc() | Reserva de la memoria para los vectores en GPU
**GPU** | cuda.memcpy_htod() | Copia el array *x_cpu* e *y_cpu* a la memoria de GPU
CPU | SourceModule() | Contiene el código del kernel
CPU | module.get_function() | Convierte el texto del kernel en una función de Python
CPU | dim_hilo, dim_bloque | Se calculan las dimensiones para la ejecución de 1D
**GPU** | kernel() | Ejecución del kernel en GPU junto con los parámetros necesarios
CPU | cuda.memcpy_dtoh() | Se copia desde la memoria GPU al CPU
CPU | loop for | Aplica el algoritmo a los vectores *x* e *y*
CPU | print() | Informa los resultados por pantalla

</center>

##5. Conclusiones
Podemos notar que el mayor tiempo de ejecución lo tiene la ejecución en el ciclo *for* de CPU, en el cual se realiza la rotación en el plano de los vectores *x* e *y*.

Esto es de esperarse, ya que es donde está el núcleo del programa que requiere más procesamiento. Cuanto más grandes sean los vectores, más tiempo de ejecución va a requerir el ciclo *for*, ya que recorre todas las posiciones, además sabiendo que este ciclo posee una complejidad O(n), implicando que mínimamente va a recorrer todos los elementos de manera secuencial.

Aún así, vemos que hay casos en los que tanto CPU como GPU podrían tardar de manera aproximada los mismos tiempos, como, por ejemplo, casos en los que los vectores tienen pocas cantidades de elementos (se ha probado con vectores de 6 elementos y se ha notado este caso en particular). De todos modos, la idea es procesar vectores de gran tamaño, ya que sería un desperdicio de HPC para dos vectores de 6 elementos cada uno.

##6. Bibliografía

* [1]: [Intel - Rutinas y funciones BLAS nivel 1: Función "rot"](https://software.intel.com/content/www/us/en/develop/documentation/mkl-developer-reference-c/top/blas-and-sparse-blas-routines/blas-routines/blas-level-1-routines-and-functions/cblas-rot.html#cblas-rot)
* [2]: [Intel - Rutinas y funciones BLAS nivel 1](https://software.intel.com/content/www/us/en/develop/documentation/mkl-developer-reference-c/top/blas-and-sparse-blas-routines/blas-routines/blas-level-1-routines-and-functions.html)
* [3]: [Colab - Guía para markdowns](https://colab.research.google.com/notebooks/markdown_guide.ipynb#scrollTo=tPqPXAKKkzaM)
* [4]: [Página oficial de CUDA para Python](https://pypi.org/project/pycuda/)
* [5]: [Documentación de CUDA](https://documen.tician.de/pycuda/index.html)
